# Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import os
import pandas as pd

# Inputs

In [ ]:
def get_user_inputs():
    search_query = input("Enter the search query: ")
    # username = input("Enter your LinkedIn username: ")
    # password = input("Enter your LinkedIn password: ")
    
    Location = input("Enter desire Location  : ")
    Company=input("Enter desire Company  : ")
    school=input("Enter desire school  : ")
    industry=input("Enter desire industry  : ")
    return search_query, Location,Company,school,industry
# search_query = get_user_inputs()
search_query,Location,Company,school,industry = get_user_inputs()


data


In [ ]:
def start():
    driver = webdriver.Chrome()
    driver.get("https://www.linkedin.com/login")
    time.sleep(10)
    username = driver.find_element(By.ID, "username")
    password = driver.find_element(By.ID, "password")

    username.send_keys("helloglowingsoft@gmail.com") 
    password.send_keys("1234REmo") 

    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    return driver


driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/login")
time.sleep(10)
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")

username.send_keys("helloglowingsoft@gmail.com") 
password.send_keys("1234REmo") 

login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
login_button.click()

In [105]:
def get_valid_email(container):
    anchor_tags = container.find_elements(By.TAG_NAME, 'a')
    for anchor in anchor_tags:
        email = anchor.text.strip()
        if email.lower().endswith('.com'):
            return email
    return None

In [104]:
def mail(driver): 
    contacts  = driver.find_element(By.XPATH, '//*[@id="top-card-text-details-contact-info"]')
    contacts.click()

    contact_ = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.artdeco-modal__content.ember-view'))
        )
    sections = contact_.find_elements(By.CSS_SELECTOR, 'section.pv-contact-info__contact-type')

    email = None
    for section in sections:

                email_container = section.find_element(By.CSS_SELECTOR, 'div.fBDirYGTiMblqxtbcqXvaYDtpieYgGANI.t-14')
                email = get_valid_email(email_container)
                if email:
                    print("Email:", email)
                else:
                       print('non')
    return email

In [106]:
def scroll_down(driver, scroll_pause_time=1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        return False
    return True

In [107]:
def collect_and_save_links(driver):
    unique_hrefs = set()
    unique_anchor_tags = []
    ancherTags = []
    counter = -1

    while True:
        try:
            ul_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.reusable-search__entity-result-list.list-style-none'))
            )
            li_elements = ul_element.find_elements(By.CSS_SELECTOR, 'li.reusable-search__result-container')

            for li_element in li_elements:
                div_element = li_element.find_element(By.CSS_SELECTOR, 'div.display-flex')
                anchor_tags_in_div = div_element.find_elements(By.CSS_SELECTOR, 'a.app-aware-link')

                for anchor_tag in anchor_tags_in_div:
                    href = anchor_tag.get_attribute('href')
                    if href and not href[-2:].upper() == "IT" and not href[-1:] == "*" and href not in unique_hrefs:
                        unique_hrefs.add(href)
                        unique_anchor_tags.append(anchor_tag)

            for anchor_tag in unique_anchor_tags:
                ancherTags.append(anchor_tag.get_attribute('href'))

            # Scroll down to load more items
            next=scroll_down(driver)
            time.sleep(3)

            # Check if the next button is available

            if not next:
                next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'button.artdeco-pagination__button--next'))
            )
                next_button.click()
                time.sleep(3)  # Adjust sleep time as necessary
                break

        except Exception as e:
                next=scroll_down(driver)
                next=scroll_down(driver)
                next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'button.artdeco-pagination__button--next'))
            )
                next_button.click()
                time.sleep(3) 
                break

    csv_file = 'urls.csv'
    header = ['URL']
    file_exists = os.path.isfile(csv_file)
    
    try:
        with open(csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                writer.writerow(header)
            for url in ancherTags:
                writer.writerow([url])
        print(f"URLs saved to {csv_file} successfully.")
    except Exception as e:
        print(f"Error saving URLs to {csv_file}: {e}")

In [108]:
def update_csv(file_path, url, name, bio, email):
    temp_file = file_path + '.tmp'
    url_found = False
    fieldnames = ['name', 'bio', 'email', 'url']

    # Check if file exists and is not empty
    file_exists = os.path.exists(file_path)
    file_empty = file_exists and os.path.getsize(file_path) == 0

    if not file_exists or file_empty:
        # Create new file with headers
        with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile, \
         open(temp_file, 'w', newline='', encoding='utf-8') as tempfile:
        reader = csv.DictReader(csvfile)
        
        # If the file has content, update fieldnames
        if reader.fieldnames:
            for field in fieldnames:
                if field not in reader.fieldnames:
                    reader.fieldnames.append(field)
            fieldnames = reader.fieldnames

        writer = csv.DictWriter(tempfile, fieldnames=fieldnames)
        writer.writeheader()

        # If file has content, process existing rows
        if reader.fieldnames:
            for row in reader:
                if row.get('url') == url:
                    row['name'] = name
                    row['email'] = email
                    row['bio'] = bio
                    
                    url_found = True
                writer.writerow(row)

        # If URL not found, add new row
        if not url_found:
            writer.writerow({'url': url, 'name': name, 'bio': bio, 'email': email})

    os.replace(temp_file, file_path)
    print(f"CSV file updated for URL: {url}")

# Write in Search BAr

In [65]:
time.sleep(5)
try:
        search_box = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
        print("Search input found directly.")
        search_box.click()

except:
        try:
            reveal_button = driver.find_element(By.XPATH, '//*[@id="global-nav-search"]/div/button')
            reveal_button.click()
            print("Clicked reveal button.")
            time.sleep(5) 
            search_box = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
            print("Search input found after revealing.")
        except Exception as e:
            print(f"Error clicking reveal button: {e}")
print(search_query)
search_box.send_keys(search_query)
search_box.send_keys(Keys.RETURN)
time.sleep(5) 

Search input found directly.
data


# Click People

In [45]:
index = 1
while index <= 20:
    button_xpath = f'//*[@id="search-reusables__filters-bar"]/ul/li[{index}]/button'
    button = driver.find_element(By.XPATH, button_xpath)
    if button.text == "People":
        button.click()
        break
    index += 1
else:
    print("Desired button not found within the specified limit.")

# Select All filter

In [7]:
def filter():
    css_selector = ".relative.mr2 .artdeco-pill.artdeco-pill--slate.artdeco-pill--choice.artdeco-pill--2.search-reusables__filter-pill-button"
    all_filters_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
    )
    all_filters_button.click()
filter()

# Select Location 

In [10]:
all_filters_panel = WebDriverWait(driver, 10).until(
   
    EC.presence_of_element_located((By.XPATH, "//div[@role='dialog']"))
)
css_selector = ".artdeco-modal__content"
content_area = all_filters_panel.find_element(By.CSS_SELECTOR, css_selector)
driver.execute_script("arguments[0].scrollTop += 150", content_area)

# Chuse Location 

In [8]:
main_div = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "artdeco-modal__content"))
)
li_element = main_div.find_elements(By.CSS_SELECTOR, "li.search-reusables__secondary-filters-filter")[4]  
buttons = driver.find_elements(By.CSS_SELECTOR, "button.artdeco-button.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view.reusable-search-filters-advanced-filters__add-filter-button")
for button in buttons:
    if button.find_element(By.XPATH, ".//span[@class='artdeco-button__text']").text == "Add a location":
        button.click()
        break
div_element = driver.find_element(By.CLASS_NAME, "search-reusables__filter-new-value-typeahead")
input_element = div_element.find_element(By.TAG_NAME, "input")

input_element.send_keys(Location) 
try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "basic-typeahead__triggered-content")))
        actions = ActionChains(driver)
        time.sleep(5)  
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()

except Exception as e:
        filter()
        filter()

In [9]:
buttons = driver.find_elements(By.CSS_SELECTOR, "button.artdeco-button.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view.reusable-search-filters-advanced-filters__add-filter-button")
for button in buttons:
    if button.find_element(By.XPATH, ".//span[@class='artdeco-button__text']").text == "Add a company":
        button.click()
        break
div_element = driver.find_element(By.CLASS_NAME, "search-reusables__filter-new-value-typeahead")
input_element = div_element.find_element(By.TAG_NAME, "input")

input_element.send_keys(Company) 
try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "basic-typeahead__triggered-content")))
        actions = ActionChains(driver)
        time.sleep(5)  
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()

except Exception as e:
        filter()
        filter()

In [11]:
buttons = driver.find_elements(By.CSS_SELECTOR, "button.artdeco-button.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view.reusable-search-filters-advanced-filters__add-filter-button")
for button in buttons:
    if button.find_element(By.XPATH, ".//span[@class='artdeco-button__text']").text == "Add an industry":
        button.click()
        break
div_element = driver.find_element(By.CLASS_NAME, "search-reusables__filter-new-value-typeahead")
input_element = div_element.find_element(By.TAG_NAME, "input")

input_element.send_keys(industry) 
try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "basic-typeahead__triggered-content")))
        actions = ActionChains(driver)
        time.sleep(5)  
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()

except Exception as e:
        filter()
        filter()

    

# Apply Filters

In [12]:
button = driver.find_element(By.CLASS_NAME, "reusable-search-filters-buttons")
button.click()

In [81]:
page_number = 5
for i in range(page_number):
    collect_and_save_links(driver)


URLs saved to urls.csv successfully.
URLs saved to urls.csv successfully.
URLs saved to urls.csv successfully.
URLs saved to urls.csv successfully.
URLs saved to urls.csv successfully.


In [109]:
def run(url,path):
    driver.execute_script("window.open(arguments[0]);", url)

    # Switch to the new tab
    driver.switch_to.window(driver.window_handles[-1])

    # Wait for the page to load
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Now search for your element
    div_element1 = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ph5'))
    )

    name_element = WebDriverWait(div_element1, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.text-heading-xlarge'))
        )
        
        # Get the text from the h1 element
    name = name_element.text
        
        # Print the name

    bio_element = WebDriverWait(div_element1, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.text-body-medium.break-words'))
        )
        
        # Get the text from the div element
    bio = bio_element.text
        
        # Print the bio

    time.sleep(10) 
    try:
        mail_=mail()
    except:
        mail_=mail()
    update_csv(path, url, name, bio, mail_)


In [110]:
file_path = 'urls.csv'
file_path_1 = 'information.csv'
df = pd.read_csv(file_path)
for url in df['URL']:
    run(url,file_path_1)


CSV file updated for URL: https://www.linkedin.com/in/zoha-javed-1b5b2919a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC7MA9kBlw35Ckj1HIfU8r7XbrEuBR_pDPY
CSV file updated for URL: https://www.linkedin.com/in/zohajaleel5?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACuUCGYBwahEE9_510Sltv9lTxzM6set3zw
CSV file updated for URL: https://www.linkedin.com/in/maryamfayyaz?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC-qK3IBbhSi2rY6zv92dL6L1newSh3Cz_U
CSV file updated for URL: https://www.linkedin.com/in/muhammad-faizan-qasim?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACzxjngBc31wG4vdd0hnB0JW57uKjgK7IvY
CSV file updated for URL: https://www.linkedin.com/in/aqib-ali-735633290?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEaSk1kBzBxB6u8Abdd2EuFVwD5UuJiJU94
CSV file updated for URL: https://www.linkedin.com/in/syedmuslim015?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAESf9N0BYj3v7f8eFXBeadkWD_lGST7XfSg
CSV file updated for URL: https://www.linkedin.com/in/syed-ali-hassan-zaidi-6